In [3]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel
from typing import Optional

model = LiteLLMModel(
    model_id="ollama_chat/llama3.2",
    api_base="http://localhost:11434", # replace with remote open-ai compatible server if necessary
    api_key="ollama" # replace with API key if necessary
)

@tool
def get_weather(location: str, celsius: Optional[bool] = False) -> str:
    """
    Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        celsius: the temperature
    """
    return "The weather is UNGODLY with torrential rains and temperatures below -10°C"

agent = ToolCallingAgent(tools=[get_weather], model=model)

print(agent.run("What's the weather like in Paris?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2 ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_weather' with arguments: {'celsius': False, 'location': 'Paris'}                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: The weather is UNGODLY with torrential rains and temperatures below -10°C

[Step 0: Duration 171.51 seconds| Input tokens: 1,170 | Output tokens: 23]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The weather in Paris is UNGODLY with torrential rains  │
│ and temperatures below -10°C.'}                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The weather in Paris is UNGODLY with torrential rains and temperatures below -10°C.

[Step 1: Duration 59.27 seconds| Input tokens: 2,460 | Output tokens: 59]

The weather in Paris is UNGODLY with torrential rains and temperatures below -10°C.


In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from smolagents import Tool, LiteLLMModel
from smolagents import CodeAgent


knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)



class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=10
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

model = LiteLLMModel(
    model_id="ollama_chat/llama3.2",
    api_base="http://localhost:11434", # replace with remote open-ai compatible server if necessary
    api_key="ollama" # replace with API key if necessary
)

retriever_tool = RetrieverTool(docs_processed)
agent = CodeAgent(
    tools=[retriever_tool], model=model, max_iterations=4, verbose=True
)

agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.2 ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To determine whether the forward or backward pass is slower for a transformer model training, we need to  
consider the computational complexity of each pass. The forward pass involves generating an output tensor from     
input data, whereas the backward pass involves computing gradients with respect to the input data.                 
                                                                                                                   
In general, the forward pass in a neural network requires less computational resources than the backward pass      
because it only needs to iterate over the input data once, applying the model's parameters and operations to       
generate an output. The backward pass, on the other hand, requires iterating over the entire sequence of           
computations (i.e., all layers and parameters) in reverse order, computing gradients using backpropagation.        
                                                                                                                   
Given this analysis, I'm going to predict that the forward pass is typically slower than the backward pass during  
transformer model training.                                                                                        
                                                                                                                   
```code                                                                                                            
# No imports or variable definitions needed for this answer                                                        
                                                                                                                   
final_answer = "forward"                                                                                           
```                                                                                                                
                                                                                                                   
Note that this prediction assumes a standard implementation of the transformer model and the PyTorch library, which
are commonly used in deep learning applications. The actual computational complexity may vary depending on specific
model architectures, optimization algorithms, and hardware configurations.                                         

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\utils.py:113 in                │
│ parse_code_blob                                                                                  │
│                                                                                                  │
│   110 │   │   pattern = r"```(?:py|python)?\n(.*?)\n```"                                         │
│   111 │   │   match = re.search(pattern, code_blob, re.DOTALL)                                   │
│   112 │   │   if match is None:                                                                  │
│ ❱ 113 │   │   │   raise ValueError(                                                              │
│   114 │   │   │   │   f"No match ground for regex pattern {pattern} in {code_blob=}."            │
│   115 │   │   │   )                                                                              │
│   116 │   │   return match.group(1).strip()                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: No match ground for regex pattern ```(?:py|python)?\n(.*?)\n``` in code_blob='Thought: To determine 
whether the forward or backward pass is slower for a transformer model training, we need to consider the 
computational complexity of each pass. The forward pass involves generating an output tensor from input data, 
whereas the backward pass involves computing gradients with respect to the input data.\n\nIn general, the forward 
pass in a neural network requires less computational resources than the backward pass because it only needs to 
iterate over the input data once, applying the model\'s parameters and operations to generate an output. The 
backward pass, on the other hand, requires iterating over the entire sequence of computations (i.e., all layers and
parameters) in reverse order, computing gradients using backpropagation.\n\nGiven this analysis, I\'m going to 
predict that the forward pass is typically slower than the backward pass during transformer model 
training.\n\n```code\n# No imports or variable definitions needed for this answer\n\nfinal_answer = 
"forward"\n```\n\nNote that this prediction assumes a standard implementation of the transformer model and the 
PyTorch library, which are commonly used in deep learning applications. The actual computational complexity may 
vary depending on specific model architectures, optimization algorithms, and hardware configurations.'.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\agents.py:912 in step          │
│                                                                                                  │
│    909 │   │                                                                                     │
│    910 │   │   # Parse                                                                           │
│    911 │   │   try:                                                                              │
│ ❱  912 │   │   │   code_action = parse_code_blob(llm_output)                                     │
│    913 │   │   except Exception as e:                                                            │
│    914 │   │   │   console.print_exception()                                                     │
│    915 │   │   │   error_msg = f"Error in code parsing: {e}. Make sure to provide correct code"  │
│                                                                                                  │
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\utils.py:119 in                │
│ parse_code_blob                                                                 

Error in code parsing: 
The code blob you used is invalid: due to the following error: No match ground for regex pattern 
```(?:py|python)?\n(.*?)\n``` in code_blob='Thought: To determine whether the forward or backward pass is slower 
for a transformer model training, we need to consider the computational complexity of each pass. The forward pass 
involves generating an output tensor from input data, whereas the backward pass involves computing gradients with 
respect to the input data.\n\nIn general, the forward pass in a neural network requires less computational 
resources than the backward pass because it only needs to iterate over the input data once, applying the model\'s 
parameters and operations to generate an output. The backward pass, on the other hand, requires iterating over the 
entire sequence of computations (i.e., all layers and parameters) in reverse order, computing gradients using 
backpropagation.\n\nGiven this analysis, I\'m going to predict that the forward pass is typically slower than the 
backward pass during transformer model training.\n\n```code\n# No imports or variable definitions needed for this 
answer\n\nfinal_answer = "forward"\n```\n\nNote that this prediction assumes a standard implementation of the 
transformer model and the PyTorch library, which are commonly used in deep learning applications. The actual 
computational complexity may vary depending on specific model architectures, optimization algorithms, and hardware 
configurations.'.
This means that the regex pattern ```(?:py|python)?\n(.*?)\n``` was not respected: make sure to include code with 
the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_action>. Make sure to provide correct code

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\utils.py:113 in                │
│ parse_code_blob                                                                                  │
│                                                                                                  │
│   110 │   │   pattern = r"```(?:py|python)?\n(.*?)\n```"                                         │
│   111 │   │   match = re.search(pattern, code_blob, re.DOTALL)                                   │
│   112 │   │   if match is None:                                                                  │
│ ❱ 113 │   │   │   raise ValueError(                                                              │
│   114 │   │   │   │   f"No match ground for regex pattern {pattern} in {code_blob=}."            │
│   115 │   │   │   )                                                                              │
│   116 │   │   return match.group(1).strip()                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: No match ground for regex pattern ```(?:py|python)?\n(.*?)\n``` in code_blob=''.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\agents.py:912 in step          │
│                                                                                                  │
│    909 │   │                                                                                     │
│    910 │   │   # Parse                                                                           │
│    911 │   │   try:                                                                              │
│ ❱  912 │   │   │   code_action = parse_code_blob(llm_output)                                     │
│    913 │   │   except Exception as e:                                                            │
│    914 │   │   │   console.print_exception()                                                     │
│    915 │   │   │   error_msg = f"Error in code parsing: {e}. Make sure to provide correct code"  │
│                                                                                                  │
│ d:\GitHub\experiment-notebooks\.venv\Lib\site-packages\smolagents\utils.py:119 in                │
│ parse_code_blob                                                                                  │
│                                                                                                  │
│   116 │   │   return match.group(1).strip()                                                      │
│   117 │                                                                                          │
│   118 │   except Exception as e:                                                                 │
│ ❱ 119 │   │   raise ValueError(                                                                  │
│   120 │   │   │   f"""                                                                           │
│   121 The code blob you used is invalid: due to the following error: {e}                         │
│   122 This means that the regex pattern {pattern} was not respected: make sure to include code   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
The code blob you used is invalid: due to the following error: No match ground for regex pattern 
```(?:py|python)?\n(.*?)\n``` in code_blob=''.
This means that the regex pattern ```(?:py|python)?\n(.*?)\n``` was not respected: make sure to include code with 
the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_action>

Error in code parsing: 
The code blob you used is invalid: due to the following error: No match ground for regex pattern 
```(?:py|python)?\n(.*?)\n``` in code_blob=''.
This means that the regex pattern ```(?:py|python)?\n(.*?)\n``` was not respected: make sure to include code with 
the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_action>. Make sure to provide correct code

[Step 3: Duration 313.37 seconds| Input tokens: 8,181 | Output tokens: 599]

Reached max iterations.